<a href="https://colab.research.google.com/github/anudeepdivi/NLP/blob/main/meddiag_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
chat_prompt = """
### Instruction:
{}


### Input:
{}


### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
   instruction = ""
   inputs       = examples["input"]
   outputs      = examples["output"]
   texts = []
   for input, output in zip(inputs, outputs):
       # Must add EOS_TOKEN, otherwise your generation will go on forever!
       text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
       texts.append(text)
   return { "text" : texts, }
pass


from datasets import load_dataset


dataset = load_dataset("bio-nlp-umass/bioinstruct", split = "train")


dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/5.02k [00:00<?, ?B/s]

biomed_instruct_25k.json:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25005 [00:00<?, ? examples/s]

Map:   0%|          | 0/25005 [00:00<?, ? examples/s]

In [5]:
import os
import re
from typing import List, Literal, Optional

from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
from datasets.builder import DatasetGenerationError


#DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

dataset = load_dataset("bio-nlp-umass/bioinstruct", split = "train")
def formatting_prompts_func(dataset):
    texts = []
    for question, answer in zip(dataset["input"], dataset["output"]):
      prompt = ""
      if tokenizer.chat_template.startswith("{% for message in messages %}"):
          # Iterate over each message (question and answer)
          prompt += "{% set messages = ["
          prompt += "{'role': 'user', 'content': '%s'}," % question
          prompt += "{'role': 'assistant', 'content': '%s'}]" % answer
          prompt += " {% endfor %}{% if add_generation_prompt is defined %}{{'<|im_start|>assistant\n' }}{% endif %}"
      texts.append(prompt)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True,)

import pprint
pprint.pprint("""NOT a formatted dataset""")
pprint
pprint.pprint(dataset[250])
pprint.pprint(dataset[260])
pprint.pprint(dataset[270])
pprint.pprint(dataset[280])
pprint.pprint(dataset[290])

# Print sample
pprint.pprint("""formatted dataset""")
pprint.pprint(dataset[250])
pprint.pprint(dataset[260])
pprint.pprint(dataset[270])
pprint.pprint(dataset[280])
pprint.pprint(dataset[290])

Map:   0%|          | 0/25005 [00:00<?, ? examples/s]

'NOT a formatted dataset'
{'input': '',
 'instruction': 'Explain the purpose of a CT scan in diagnosing medical '
                'conditions.',
 'output': 'A CT scan is an imaging technique used to create detailed pictures '
           "of the body's internal structures. It can help diagnose medical "
           'conditions, detect abnormalities, monitor treatments, and guide '
           'certain medical procedures by providing detailed and accurate '
           'information about organs, tissues, blood vessels, and bones.',
 'text': ''}
{'input': '',
 'instruction': 'Explain the difference between a false positive and a false '
                'negative result in medical testing.',
 'output': 'A false positive result occurs when a test mistakenly indicates '
           'the presence of a condition, such as a disease, when it is not '
           'actually present. A false negative result occurs when a test '
           'mistakenly indicates the absence of a condition, such as a '
   

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 35,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 35
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,16.096500
2,16.096500
3,15.709500
4,14.389400
5,12.293800
6,10.025300
7,8.321800
8,7.844700
9,7.287500
10,6.553800


###DIAGNOSIS INFERENCE###

In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    chat_prompt.format(
        "", # instruction
        "A 40-year-old female presents with persistent cough, shortness of breath, fatigue, and occasional chest pain. She has a history of smoking and reports recent unexplained weight loss. Based on these symptoms, suggest possible medical conditions and recommend further diagnostic steps.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>\n### Instruction:\n\n\n\n### Input:\nA 40-year-old female presents with persistent cough, shortness of breath, fatigue, and occasional chest pain. She has a history of smoking and reports recent unexplained weight loss. Based on these symptoms, suggest possible medical conditions and recommend further diagnostic steps.\n\n\n### Response:\nBased on the patient's symptoms, possible medical conditions that may be considered are:\n\n1.  **Chronic Obstructive Pulmonary Disease (COPD):** Given her history of smoking and symptoms such as persistent cough, shortness of breath, and fatigue, COPD is a likely possibility. However, COP"]

In [12]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

###SUMMARIZATION INFERENCE###

In [35]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    chat_prompt.format(
        "", # instruction
"You are an expert medical writer. Summarize the following research study into a concise and clear summary, including its purpose, methodology, key findings, and conclusion. Study Details: Title: The Effectiveness of Telemedicine in Managing Chronic Diseases Background: Telemedicine has become increasingly important for remote healthcare delivery, especially during the COVID-19 pandemic. Purpose: To evaluate the effectiveness of telemedicine compared to in-person care for managing chronic diseases like diabetes and hypertension. Methodology: A meta-analysis of 25 studies conducted between 2015 and 2022, with a combined sample size of 10,000 patients. Outcomes measured included blood pressure, HbA1c levels, and patient satisfaction. Key Findings: Telemedicine was found to be as effective as in-person care in improving blood pressure and HbA1c levels, with high levels of patient satisfaction reported. Conclusion: Telemedicine is a viable alternative to in-person care for chronic disease management, with potential to improve access and reduce healthcare costs.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>\n### Instruction:\n\n\n\n### Input:\nYou are an expert medical writer. Summarize the following research study into a concise and clear summary, including its purpose, methodology, key findings, and conclusion. Study Details: Title: The Effectiveness of Telemedicine in Managing Chronic Diseases Background: Telemedicine has become increasingly important for remote healthcare delivery, especially during the COVID-19 pandemic. Purpose: To evaluate the effectiveness of telemedicine compared to in-person care for managing chronic diseases like diabetes and hypertension. Methodology: A meta-analysis of 25 studies conducted between 2015 and 2022, with a combined sample size of 10,000 patients. Outcomes measured included blood pressure, HbA1c levels, and patient satisfaction. Key Findings: Telemedicine was found to be as effective as in-person care in improving blood pressure and HbA1c levels, with high levels of patient satisfaction reported. Conclusion: Telemedicine is a v

###TEXT GENERATION INFERENCE###

In [36]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    chat_prompt.format(
        "", # instruction
"explain about parkinson's disease", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>\n### Instruction:\n\n\n\n### Input:\nexplain about parkinson's disease\n\n\n### Response:\nParkinson's disease is a progressive neurological disorder that affects movement, balance, and coordination. It is characterized by the degeneration of dopamine-producing neurons in the brain, leading to a deficiency of dopamine in the brain. This deficiency causes a range of symptoms, including:\n\n*   **Tremors**: Shaking or trembling"]